In [1]:
import pandas as pd
import os

caminho = 'C:\\Users\\Edmilson\\Gama_Workspace2\\ABMS-WP'

# 1. Carregar os dados
# Carregar o Tabela_consumidores_Itapua_com_setor.csv com SK_MATRICULA e NN_MORADORES
arquivo_df = pd.read_csv(os.path.join(caminho,'includes\\Tabela_consumidores_Itapua_com_setor.csv'), sep=',')

# Carregar o arquivo Tabela_consumo_Itapua_12m.csv
consumo_df = pd.read_csv(os.path.join(caminho,'includes\\Tabela_consumo_Itapua_60m.csv'), sep=';')

# 2. Calcular a média de consumo dos últimos 6 meses para cada SK_MATRICULA
# Converter AM_REFERENCIA para datetime para facilitar a filtragem
consumo_df['AM_REFERENCIA'] = pd.to_datetime(consumo_df['AM_REFERENCIA'], format='%Y%m')

# Ordenar por SK_MATRICULA e AM_REFERENCIA
consumo_df = consumo_df.sort_values(by=['SK_MATRICULA', 'AM_REFERENCIA'], ascending=[True, False])

# Filtrar os últimos 12 meses para cada SK_MATRICULA
ultimos_12_meses = consumo_df.groupby('SK_MATRICULA').head(12)

# Calcular a média de consumo (NN_MEDIA_CONSUMO) para cada SK_MATRICULA
media_consumo = ultimos_12_meses.groupby('SK_MATRICULA')['HCLQTCON'].mean().reset_index()
media_consumo.rename(columns={'HCLQTCON': 'NN_MEDIA_CONSUMO'}, inplace=True)


In [2]:

# 3. Juntar a média de consumo ao arquivo.csv
arquivo_df = pd.merge(arquivo_df, media_consumo, on='SK_MATRICULA', how='left')

In [3]:
# Primeiro, remover registros onde CD_SETOR é nulo ou vazio
arquivo_df = arquivo_df.dropna(subset=['CD_SETOR'])  # Remove registros com CD_SETOR nulo
arquivo_df = arquivo_df[arquivo_df['CD_SETOR'].astype(str) != '']  # Remove registros com CD_SETOR vazio

In [4]:
# Extrair os últimos 3 caracteres da coluna CD_SETOR
arquivo_df['CD_SETOR'] = 'S' + arquivo_df['CD_SETOR'].astype(str).str[-5:-2]

In [5]:


# 4. Classificar o comportamento (AMBIENTALISTA, NORMAL ou PERDULÁRIO)
# AMBIENTALISTA < 130
# NORMAL > 130 E <= 160
# PERDULARIO > 160
arquivo_df['NN_CONSUMO_DIARIO'] = ((arquivo_df['NN_MEDIA_CONSUMO']*1000 / arquivo_df['NN_MORADORES']) / 30.5)
arquivo_df['TP_COMPORTAMENTO'] = arquivo_df['NN_CONSUMO_DIARIO'].apply(
    lambda consumo: 
                   'AMBIENTALISTA' if consumo < 130 
                   else 'MODERADO' if 130 <= consumo <= 160 
                   else 'PERDULARIO'
)


# 5. Salvar o arquivo.csv com a nova coluna TP_COMPORTAMENTO
arquivo_df.to_csv(os.path.join(caminho,'includes\\Tabela_consumidores_Itapua_com_setor_e_comportamento.csv'), sep=';',index=False)

# 6. Visualizar o resultado
print(arquivo_df.head())

   SK_MATRICULA NM_SUBCATEGORIA  NN_MORADORES  ST_PISCINA    LAT_GEO  \
0     124584355          NORMAL             5           0 -12.950571   
1     152961499          NORMAL             1           0 -12.943031   
2     125305257   INTERMEDIARIA             8           0 -12.944554   
3     123863460          NORMAL             3           0 -12.940758   
4     126026156          NORMAL             2           0 -12.936883   

    LONG_GEO              X             Y CD_SETOR  NN_MEDIA_CONSUMO  \
0 -38.373685  567933.093052  8.568247e+06     S305          3.916667   
1 -38.366573  568706.588075  8.569079e+06     S291          8.583333   
2 -38.363644  569023.883741  8.568910e+06     S067          8.250000   
3 -38.369058  568437.655084  8.569331e+06     S275         23.083333   
4 -38.358508  569583.120489  8.569757e+06     S106          5.833333   

   NN_CONSUMO_DIARIO TP_COMPORTAMENTO  
0          25.683060    AMBIENTALISTA  
1         281.420765       PERDULARIO  
2          33.

In [6]:
# Contagem de AMBIENTALISTAS, NORMAL e PERDULÁRIOS
contagem_comportamento = arquivo_df['TP_COMPORTAMENTO'].value_counts()

# Percentual em relação ao total
percentual_comportamento = arquivo_df['TP_COMPORTAMENTO'].value_counts(normalize=True) * 100

# Combinar os resultados em um único DataFrame
resultado = pd.DataFrame({
    'Contagem': contagem_comportamento,
    'Percentual (%)': percentual_comportamento
})

# Exibir o resultado
print(resultado)

                  Contagem  Percentual (%)
TP_COMPORTAMENTO                          
PERDULARIO            9205       60.175198
AMBIENTALISTA         5330       34.843433
MODERADO               762        4.981369
